### **Converting to Mono Audio Dataset**

In [12]:
# !git clone https://github.com/oridarshan/FallKol.git

In [13]:
# %pip install pydub

In [14]:
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, Gain, PolarityInversion, ClippingDistortion, TimeMask, LoudnessNormalization, Resample, HighPassFilter, LowPassFilter, BandPassFilter
from pydub import AudioSegment
import os
import numpy as np

In [15]:
# def stereo_to_mono(audio_file_name, output_file_name):
#     if audio_file_name.endswith('.wav'):
#         sound = AudioSegment.from_wav(audio_file_name)
#         sound = sound.set_channels(1)
#         sound.export(output_file_name, format="wav")

In [16]:
import warnings
import random

def augment_audio_fall(audio_file_name, output_file_name):
    
    try:
        augment = Compose([
            AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
            Gain(min_gain_in_db=-12, max_gain_in_db=12, p=0.5),
            TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
            Shift(min_shift=-0.5, max_shift=0.5, p=0.5),
        ])


      # 'Couldn't find ffprobe or avprobe' 경고 무시
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=RuntimeWarning)
            audio = AudioSegment.from_wav(audio_file_name)

            samples = np.array(audio.get_array_of_samples())
            sample_rate = audio.frame_rate

            augmented_samples = augment(samples=samples.astype(np.float32), sample_rate=sample_rate)

            # 데이터를 적절하게 잘라서 sample_width * channels의 배수가 되도록 합니다.
            total_samples = len(augmented_samples)
            samples_per_frame = audio.sample_width * audio.channels
            remaining_samples = total_samples % samples_per_frame
            if remaining_samples != 0:
                augmented_samples = augmented_samples[:-remaining_samples]

            augmented_audio = AudioSegment(data=augmented_samples.tobytes(), frame_rate=sample_rate, sample_width=audio.sample_width, channels=audio.channels)

            augmented_audio.export(output_file_name, format="wav")

    except Exception as e:
        print(f"오디오 파일 {audio_file_name} 처리 중 오류 발생: {e}")
        return

In [17]:
def augment_audio_nonfall(audio_file_name, output_file_name):
    # 랜덤한 파라미터 값 생성
    
    try:
        augment = Compose([
            AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
            Gain(min_gain_in_db=-12, max_gain_in_db=12, p=0.5),
            LoudnessNormalization(min_lufs_in_db=-31, max_lufs_in_db=-13, p=0.5),
            PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
            Resample(min_sample_rate=8000, max_sample_rate=44100, p=0.5),
            TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
            Shift(min_shift=-0.5, max_shift=0.5, p=0.5),
            HighPassFilter(min_cutoff_freq=20, max_cutoff_freq=24000, p=0.5),
            LowPassFilter(min_cutoff_freq=20, max_cutoff_freq=24000, p=0.5),
            BandPassFilter(min_center_freq=20, max_center_freq=24000, p=0.5),
            ClippingDistortion(min_percentile_threshold=0, max_percentile_threshold=1, p=0.5),
            PolarityInversion(p=0.5),
            TimeMask(min_band_part=0.0, max_band_part=0.2, p=0.5)
        ])


      # 'Couldn't find ffprobe or avprobe' 경고 무시
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=RuntimeWarning)
            audio = AudioSegment.from_wav(audio_file_name)

            samples = np.array(audio.get_array_of_samples())
            sample_rate = audio.frame_rate

            augmented_samples = augment(samples=samples.astype(np.float32), sample_rate=sample_rate)

            # 데이터를 적절하게 잘라서 sample_width * channels의 배수가 되도록 합니다.
            total_samples = len(augmented_samples)
            samples_per_frame = audio.sample_width * audio.channels
            remaining_samples = total_samples % samples_per_frame
            if remaining_samples != 0:
                augmented_samples = augmented_samples[:-remaining_samples]

            augmented_audio = AudioSegment(data=augmented_samples.tobytes(), frame_rate=sample_rate, sample_width=audio.sample_width, channels=audio.channels)

            augmented_audio.export(output_file_name, format="wav")

    except Exception as e:
        print(f"오디오 파일 {audio_file_name} 처리 중 오류 발생: {e}")
        return

In [18]:
# # Define the directory paths
# file_path = 'Audio_Files'
# fall_output_path = 'Audio_Files/Fall_Mono'
# notfall_output_path = 'Audio_Files/Non_Fall_Mono'

In [19]:
# Define input directories for fall and nonfall files
fall_input_path = 'Audio_Files/Fall_Mono'
nonfall_input_path = 'Audio_Files/Non_Fall_Mono'

# Define output directories for fall and nonfall files
fall_output_path = 'Audio_Files/Fall_Mono'
nonfall_output_path = 'Audio_Files/Non_Fall_Mono'

In [20]:
fall_augmented_output_path = 'Aug_Audio_Files\Aug_Fall_Mono'
nonfall_augmented_output_path = 'Aug_Audio_Files/Aug_NonFall_Mono'

In [21]:
# 폴 오디오 증강
cnt_fall = 1
for i in range(10):
    for file_name in os.listdir(fall_input_path):
        full_file_name = os.path.join(fall_input_path, file_name)
        augmented_output_file_name = os.path.join(fall_augmented_output_path, f'fall_aug_{cnt_fall}.wav')

        augment_audio_fall(full_file_name, augmented_output_file_name)
        cnt_fall += 1

In [22]:
# 논폴 오디오 증강
cnt_nonfall = 1
for i in range(10):
    for file_name in os.listdir(nonfall_input_path):
        full_file_name = os.path.join(nonfall_input_path, file_name)
        augmented_output_file_name = os.path.join(nonfall_augmented_output_path, f'notfall_aug_{cnt_nonfall}.wav')

        augment_audio_nonfall(full_file_name, augmented_output_file_name)
        cnt_nonfall += 1

c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped s

오디오 파일 Audio_Files/Non_Fall_Mono\notfall134.wav 처리 중 오류 발생: [WinError 2] 지정된 파일을 찾을 수 없습니다


c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


오디오 파일 Audio_Files/Non_Fall_Mono\notfall162.wav 처리 중 오류 발생: [WinError 2] 지정된 파일을 찾을 수 없습니다


c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped s

...

c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped s

오디오 파일 Audio_Files/Non_Fall_Mono\notfall134.wav 처리 중 오류 발생: [WinError 2] 지정된 파일을 찾을 수 없습니다


c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


오디오 파일 Audio_Files/Non_Fall_Mono\notfall162.wav 처리 중 오류 발생: [WinError 2] 지정된 파일을 찾을 수 없습니다


c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
